<a target="_blank" href="https://colab.research.google.com/github/FranQuant/the_ai_engineer_capstones/blob/main/capstones/Week02_backprop/01_numpy_manual.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# 01 — Manual NumPy Backprop (1-Hidden-Layer MLP)

This notebook implements:

1. A tiny synthetic dataset  
2. Manual forward pass (NumPy)  
3. Manual backward pass via chain rule  
4. Gradient check (finite differences)

Following the *Deep Learning Basics: Chain Rule → Backprop → nn.Module* handout (TAE Program, Week 2).

## 1. Imports & Deterministic Seeds

In [1]:
import numpy as np

# ------------------------------
# Deterministic seed
# ------------------------------
SEED = 42
rng = np.random.default_rng(SEED) 
                                
def set_seed(seed=42):
    global rng
    rng = np.random.default_rng(seed)

set_seed(SEED)

print("NumPy RNG initialized with seed =", SEED)

NumPy RNG initialized with seed = 42


## 2. Synthetic Dataset
We implement the XOR-style pattern:

$$
 x ~ U([-1, 1]^2), y = 1[x1 * x2 < 0]
$$

The dataset construction logic follows the standard XOR quadrant logic.

- Each sample is drawn uniformly from the square  
  $$[-1, 1]^2.$$

- The binary label is generated using the rule  
  $$
  y = \mathbf{1}[\,x_1 x_2 < 0\,],
  $$
  which captures the XOR pattern  
  (points where the two coordinates have opposite signs).


- Shapes:
  - Feature matrix  
    $$X \in \mathbb{R}^{n_{\text{samples}} \times 2}$$
  - Label vector  
    $$y \in \mathbb{R}^{n_{\text{samples}}}$$   

In [2]:
# ---------------------------------------
# Synthetic XOR-like dataset
# ---------------------------------------
# According to Section 6.1 of the Capstone:
#   x ~ Uniform([-1, 1]^2)
#   y = 1[x1 * x2 < 0]

def generate_toy_data(n_samples=64):
    X = rng.uniform(-1, 1, size=(n_samples, 2))
    y = (X[:, 0] * X[:, 1] < 0).astype(float)
    return X, y

X, y = generate_toy_data(4)
print("X sample:\n", X)
print("y sample:\n", y)

X sample:
 [[ 0.5479121  -0.12224312]
 [ 0.71719584  0.39473606]
 [-0.8116453   0.9512447 ]
 [ 0.5222794   0.57212861]]
y sample:
 [1. 0. 1. 0.]


## 3. Model Parameters (NumPy)
Parameter initializations:

- $W_1$: shape $(h, d)$  
- $b_1$: shape $(h,)$  
- $W_2$: shape $(1, h)$  
- $b_2$: shape $(1,)$  

These are mathematically consistent with a 1-hidden-layer MLP:

- Input vector  
  $$x \in \mathbb{R}^2$$

- First affine layer  
  $$
  W_1 x + b_1 \in \mathbb{R}^h
  $$

- Activation  
  $$
  h = \text{ReLU}(W_1 x + b_1)
  $$

- Output layer  
  $$
  W_2 h + b_2 \in \mathbb{R}
  $$

In [3]:
# ---------------------------------------
# Model parameters: d -> h -> 1
# ---------------------------------------

d = 2     # input dimension
h = 4     # hidden width
out = 1   # scalar output

# small Gaussian init to avoid saturation
W1 = rng.normal(0, 0.1, size=(h, d))
b1 = np.zeros((h,))

W2 = rng.normal(0, 0.1, size=(1, h))
b2 = np.zeros((1,))

def get_params():
    return W1, b1, W2, b2

print("W1:", W1.shape)
print("b1:", b1.shape)
print("W2:", W2.shape)
print("b2:", b2.shape)

W1: (4, 2)
b1: (4,)
W2: (1, 4)
b2: (1,)


## 4. Activation Functions (ReLU + ReLU')
We use the Rectified Linear Unit:

$$
\mathrm{ReLU}(u) = \max(0, u),
$$

applied elementwise.  
Its derivative is

$$
\mathrm{ReLU}'(u) =
\begin{cases}
1 & \text{if } u > 0, \\
0 & \text{otherwise}.
\end{cases}
$$

Note: ReLU is **non-differentiable at $u = 0$**; this will introduce subtle
issues during numerical gradient checking near zero-activation units.

In [4]:
# ---------------------------------------
# Activation: ReLU and ReLU'
# ---------------------------------------

def relu(u):
    """
    Elementwise ReLU.
    Input:
        u : array-like (any shape)
    Returns:
        out : same shape as u
    """
    return np.maximum(0, u)


def relu_prime(u):
    """
    Elementwise derivative of ReLU.
    Returns 1 where u > 0, else 0.

    Note:
    The derivative is undefined at u == 0; we follow the standard
    subgradient convention used in deep learning frameworks.
    """
    return (u > 0).astype(float)

## 5. Forward Pass (Single Sample)
For a single input $ x \in \mathbb{R}^d $, the 1-hidden-layer MLP computes:

1. First affine layer  
$$
a_1 = W_1 x + b_1 \quad\in \mathbb{R}^h
$$

2. Activation  
$$
h = \phi(a_1) = \mathrm{ReLU}(a_1)
$$

3. Output layer  
$$
f = W_2 h + b_2 \quad\in \mathbb{R}
$$

We return the intermediate values to facilitate backpropagation.

In [5]:
# ---------------------------------------
# Forward pass: single sample
# ---------------------------------------

def forward_single(x, W1, b1, W2, b2):
    """
    Forward pass for one input sample.

    Inputs:
        x  : (d,)      input vector
        W1 : (h, d)
        b1 : (h,)
        W2 : (1, h)
        b2 : (1,)

    Returns:
        a1 : (h,)      pre-activation
        h  : (h,)      hidden layer activation (ReLU)
        f  : float     scalar model output
    """
    a1 = W1 @ x + b1        # shape (h,)
    h  = relu(a1)           # shape (h,)
    f  = W2 @ h + b2        # shape (1,), array
    return a1, h, f.item()  

## 6. Loss Function
We use the scalar squared-error loss

$$
L(f, y) = \frac{1}{2}(f - y)^2.
$$

It is differentiable with respect to the scalar model output $f$, and  
its derivative will appear in the backward pass:

$$
\frac{\partial L}{\partial f} = f - y.
$$


In [6]:
# ---------------------------------------
# Loss function: scalar squared error
# ---------------------------------------

def loss_fn(f_scalar, y_scalar):
    """
    Compute L = 0.5 * (f - y)^2 for scalar f, y.
    
    Inputs:
        f_scalar : float
        y_scalar : float
    Returns:
        L        : float
    """
    return 0.5 * (f_scalar - y_scalar) ** 2

def loss_grad_f(f_scalar, y_scalar):
    """
    Derivative dL/df = (f - y).
    """
    return f_scalar - y_scalar

## 7. Manual Backward Pass (Chain Rule)

We apply the chain rule to compute the gradients of the loss
$$
L = \tfrac12 (f - y)^2
$$
with respect to all parameters.

**Output layer**
$$
\delta_f = f - y,
\qquad
\frac{\partial L}{\partial W_2} = \delta_f \, h^\top,
\qquad
\frac{\partial L}{\partial b_2} = \delta_f.
$$

**Hidden layer**
$$
\delta_h = W_2^\top \, \delta_f,
\qquad
\delta_{a_1} = \delta_h \odot \phi'(a_1).
$$

**Input layer**
$$
\frac{\partial L}{\partial W_1} = \delta_{a_1} \, x^\top,
\qquad
\frac{\partial L}{\partial b_1} = \delta_{a_1}.
$$

All gradient arrays match the shapes of their corresponding parameters.

In [7]:
# ---------------------------------------
# Backward pass: single sample (manual)
# ---------------------------------------

def backward_single(x, y, a1, h, f, W1, b1, W2, b2):
    """
    Manual backward pass using the chain rule.

    Inputs:
        x  : (d,)
        y  : float
        a1 : (h,)
        h  : (h,)
        f  : float
        W1 : (h, d)
        b1 : (h,)
        W2 : (1, h)
        b2 : (1,)

    Returns:
        dW1 : (h, d)
        db1 : (h,)
        dW2 : (1, h)
        db2 : (1,)
    """

    # ----- output layer -----
    df = f - y                      # scalar (dL/df)

    dW2 = df * h[None, :]           # (1, h) correctly matches W2
    db2 = np.array([df])            # (1,) matches b2

    # ----- hidden layer -----
    dh  = W2[0] * df                # W2[0] is (h,) -> produces (h,)
    da1 = dh * relu_prime(a1)       # (h,)

    # ----- input layer -----
    dW1 = da1[:, None] @ x[None, :] # (h, d)
    db1 = da1                       # (h,)

    return dW1, db1, dW2, db2

## 8. Single-Sample Sanity Check

Before performing numerical gradient checking, we verify that the
manual forward and backward passes run end-to-end without shape or
broadcasting errors.

For a single data point $(x_0, y_0)$, we compute:

- forward activations and loss,
- all parameter gradients from the backward pass,
- and confirm that each gradient has the expected shape.

In [8]:
# ---------------------------------------
# Sanity check: forward + backward (single sample)
# ---------------------------------------

# pick first sample
x0, y0 = X[0], y[0]

# forward pass
a1, h, f = forward_single(x0, W1, b1, W2, b2)
L = loss_fn(f, y0)

# backward pass
dW1, db1, dW2, db2 = backward_single(x0, y0, a1, h, f, W1, b1, W2, b2)

# print diagnostics
print("Forward output f =", f)
print("Loss L =", L)
print("dW1 shape:", dW1.shape)
print("db1 shape:", db1.shape)
print("dW2 shape:", dW2.shape)
print("db2 shape:", db2.shape, "value:", db2)

Forward output f = -0.0035382554995001636
Loss L = 0.5035445151254901
dW1 shape: (4, 2)
db1 shape: (4,)
dW2 shape: (1, 4)
db2 shape: (1,) value: [-1.00353826]


## 9. Gradient Check (Finite Differences)

### 9.1 Parameter Flattening and Unflattening

To perform numerical gradient checking, we need to flatten all model
parameters into a single vector `theta`, perturb coordinates one by one,
and reconstruct the parameter tensors from the perturbed vector.

A robust implementation must:
- preserve the **exact original shapes** of each tensor,
- avoid hardcoded dimensions (`d`, `h`),
- avoid silent shape mismatches,
- ensure `b2` always has shape `(1,)`.

We therefore implement dimension-agnostic flatten/unflatten utilities.

In [9]:
# ---------------------------------------
# Utilities: flatten/unflatten parameters
# ---------------------------------------

def flatten_params(W1, b1, W2, b2):
    """
    Flatten all parameters into a single 1D array.
    Parameters must already have shapes:
        W1 : (h, d)
        b1 : (h,)
        W2 : (1, h)
        b2 : (1,)
    """
    return np.concatenate([
        W1.reshape(-1),
        b1.reshape(-1),
        W2.reshape(-1),
        b2.reshape(-1),
    ])


def unflatten_params(theta, W1_shape, b1_shape, W2_shape, b2_shape):
    """
    Reconstruct parameters from a flattened theta vector.

    Shapes must be passed explicitly to avoid hardcoded dimensions.
    """
    # Compute sizes
    size_W1 = np.prod(W1_shape)
    size_b1 = np.prod(b1_shape)
    size_W2 = np.prod(W2_shape)
    size_b2 = np.prod(b2_shape)

    # Slicing indices
    i0 = 0
    i1 = i0 + size_W1
    i2 = i1 + size_b1
    i3 = i2 + size_W2
    i4 = i3 + size_b2  # final index (redundant, but clean)

    # Unflatten
    W1 = theta[i0:i1].reshape(W1_shape)
    b1 = theta[i1:i2].reshape(b1_shape)
    W2 = theta[i2:i3].reshape(W2_shape)
    b2 = theta[i3:i4].reshape(b2_shape)

    return W1, b1, W2, b2

### 9.2 Loss Wrapper for a Single Sample

For numerical gradient checking, we require a function

$$
\ell(\theta) = L(f(\theta; x), y)
$$

that takes a flattened parameter vector `theta`, reconstructs the model
parameters, runs a forward pass on a single sample `(x, y)`, and returns
the scalar loss. This isolates the dependence on `theta` and enables
finite-difference perturbations for each parameter coordinate.

In [10]:
# ---------------------------------------
# Loss wrapper for a flattened parameter vector
# ---------------------------------------

def loss_from_theta(theta, x, y,
                    W1_shape, b1_shape,
                    W2_shape, b2_shape):
    """
    Compute L(theta) for a single sample (x, y):

        1. Unflatten theta back into W1, b1, W2, b2
        2. Forward pass to compute f
        3. Return scalar loss L = 0.5 * (f - y)^2

    Shapes are passed explicitly to avoid hardcoded dimensions.
    """
    W1, b1, W2, b2 = unflatten_params(
        theta,
        W1_shape, b1_shape,
        W2_shape, b2_shape
    )
    
    a1, h, f = forward_single(x, W1, b1, W2, b2)
    return loss_fn(f, y)

### 9.3 Analytic Gradient (Flattened)

To compare analytic and numerical gradients, we need the analytic gradient
expressed as a single flattened vector. This requires:

1. Unflattening `theta` into `(W1, b1, W2, b2)`
2. Computing all parameter gradients via `backward_single`
3. Flattening the resulting gradients back into a 1D vector

All tensor shapes are passed explicitly to avoid hardcoded assumptions.

In [11]:
# ---------------------------------------
# Analytic gradient, flattened
# ---------------------------------------

def analytic_grad(theta, x, y,
                  W1_shape, b1_shape,
                  W2_shape, b2_shape):
    """
    Compute the analytic gradient dL/dtheta as a flattened 1D vector.

    Steps:
    1. Unflatten theta -> W1, b1, W2, b2
    2. Forward pass to compute activations
    3. Backward pass to compute gradients
    4. Flatten gradients back into a single vector
    """
    # unflatten with explicit shapes
    W1, b1, W2, b2 = unflatten_params(
        theta,
        W1_shape, b1_shape,
        W2_shape, b2_shape
    )

    # forward + backward
    a1, h, f = forward_single(x, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = backward_single(
        x, y, a1, h, f, W1, b1, W2, b2
    )

    # flatten gradients (db2 already shape (1,))
    return flatten_params(dW1, db1, dW2, db2)

### 9.4 Numerical Gradient (Central Differences)

We approximate each partial derivative of the loss with respect to the
flattened parameter vector `theta` using the centered finite-difference:

$$
\frac{\partial L}{\partial \theta_i} \approx
\frac{L(\theta + \varepsilon e_i) -
      L(\theta - \varepsilon e_i)}{2\varepsilon}.
$$

Due to ReLU’s non-differentiability at zero, coordinates that place any
hidden pre-activation \(a_1\) exactly at zero can yield noisy numerical
gradients; we skip such coordinates during checking.


In [12]:
# ---------------------------------------
# Numerical gradient (central differences)
# ---------------------------------------

def numeric_grad(theta, x, y,
                 W1_shape, b1_shape,
                 W2_shape, b2_shape,
                 eps=1e-5):
    """
    Compute numerical gradient of L(theta) via central differences.

    Inputs:
        theta : (P,) flattened parameter vector
        x, y  : single sample
        eps   : finite-difference step size
        shape arguments : used for unflattening

    Returns:
        num_grads : (P,) numerical gradient
    """
    num_grads = np.zeros_like(theta)
    theta_plus  = theta.copy()
    theta_minus = theta.copy()
    
    for i in range(len(theta)):
        # finite-difference perturbations
        theta_plus[i]  = theta[i] + eps
        theta_minus[i] = theta[i] - eps

        # compute losses
        f_plus = loss_from_theta(
            theta_plus, x, y,
            W1_shape, b1_shape,
            W2_shape, b2_shape
        )
        f_minus = loss_from_theta(
            theta_minus, x, y,
            W1_shape, b1_shape,
            W2_shape, b2_shape
        )

        num_grads[i] = (f_plus - f_minus) / (2 * eps)

        # restore coordinates
        theta_plus[i]  = theta[i]
        theta_minus[i] = theta[i]

    return num_grads

### 9.5 Gradient Check Comparison

We now compare the analytic gradient vector with the numerical finite-difference
gradient. For well-behaved (non-ReLU-boundary) coordinates, the maximum absolute
difference should be on the order of machine precision (≈1e−6 to 1e−7).

In [13]:
# ---------------------------------------
# Gradient check: analytic vs numeric
# ---------------------------------------

# pick sample
x0, y0 = X[0], y[0]

# store parameter shapes
W1_shape, b1_shape = W1.shape, b1.shape
W2_shape, b2_shape = W2.shape, b2.shape

# flatten initial parameters
theta = flatten_params(W1, b1, W2, b2)

# compute analytic and numeric gradients
g_analytic = analytic_grad(
    theta, x0, y0,
    W1_shape, b1_shape,
    W2_shape, b2_shape
)

g_numeric = numeric_grad(
    theta, x0, y0,
    W1_shape, b1_shape,
    W2_shape, b2_shape
)

# diagnostics
abs_diff = np.abs(g_analytic - g_numeric)
rel_diff = abs_diff / (np.abs(g_numeric) + 1e-8)

print("Max absolute diff: ", abs_diff.max())
print("Max relative diff: ", rel_diff.max())

# Helpful diagnostic: show top offending coordinates
top_k = np.argsort(abs_diff)[-5:]
print("\nTop-5 largest coordinate diffs:")
for idx in top_k:
    print(f"  idx {idx}: analytic={g_analytic[idx]:.6f}, numeric={g_numeric[idx]:.6f}, abs diff={abs_diff[idx]:.6e}")

# fail loudly if wrong (skip extremely small ReLU-boundary anomalies)
assert abs_diff.max() < 1e-5, "Gradient check FAILED!"
print("\nGradient check PASSED.")

Max absolute diff:  9.519570028093671e-12
Max relative diff:  8.721610611055463e-09

Top-5 largest coordinate diffs:
  idx 9: analytic=0.096228, numeric=0.096228, abs diff=7.995535e-12
  idx 13: analytic=-0.038812, numeric=-0.038812, abs diff=8.153749e-12
  idx 0: analytic=-0.020276, numeric=-0.020276, abs diff=8.851111e-12
  idx 12: analytic=-0.009541, numeric=-0.009541, abs diff=9.498794e-12
  idx 11: analytic=0.005010, numeric=0.005010, abs diff=9.519570e-12

Gradient check PASSED.
